## Start HumemAI

In [1]:
from humemai.utils import disable_logger
disable_logger()

from gremlin_python.structure.graph import Graph, Vertex, Edge
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver.serializer import GraphSONSerializersV3d0
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.traversal import P, T, Direction

import json
from humemai.janusgraph import Humemai
from tqdm.auto import tqdm

humemai = Humemai()
humemai.start_containers()
humemai.connect()
# humemai.remove_all_data()


with open("./raw-data.json") as f:
    data = json.load(f)

# humemai.disconnect()
# humemai.stop_containers()
# humemai.remove_containers()

from humemai.utils import disable_logger
disable_logger()

/home/tk/.virtualenvs/humemai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
DEBUG:docker.utils.config:Trying paths: ['/home/tk/.docker/config.json', '/home/tk/.dockercfg']
DEBUG:docker.utils.config:No config file found
DEBUG:docker.utils.config:Trying paths: ['/home/tk/.docker/config.json', '/home/tk/.dockercfg']
DEBUG:docker.utils.config:No config file found
DEBUG:urllib3.connectionpool:http://localhost:None "GET /version HTTP/11" 200 834
DEBUG:docker.utils.config:Trying paths: ['/home/tk/.docker/config.json', '/home/tk/.dockercfg']
DEBUG:docker.utils.config:No config file found
DEBUG:docker.utils.config:Trying paths: ['/home/tk/.docker/config.json', '/home/tk/.dockercfg']
DEBUG:docker.utils.config:No config file found
DEBUG:urllib3.connectionpool:http://localhost:None "GET /version HTTP/11" 200 834


In [ ]:
humemai.remove_all_data()

for data_point in tqdm(data):
    properties = {
        "cp_num": data_point["cp_num"],
        "participant_num": data_point["participant"],
        "cp_name": data_point["cp_name"],
        "ticks_lasted": data_point["ticks_lasted"],
        "round_num": data_point["round_num"],
        "event_time": data_point["timestamp"],
        "remaining_time": data_point["remaining_time"],
        "remaining_rocks": data_point["remaining_rocks"],
        "victim_harm": data_point["victim_harm"],
    }
    time_vertex = humemai.write_time_vertex(data_point["timestamp"])

    robot_vertex = humemai.write_long_term_vertex("robot", "episodic", time_vertex)
    cp_label = "cp_num_" + str(data_point["cp_num"])
    cp_vertex = humemai.write_long_term_vertex(
        cp_label, "episodic", time_vertex, properties
    )
    humemai.write_edge(robot_vertex, "has_cp", cp_vertex)

    participant_label = "participant_" + str(data_point["participant"])
    participant_vertex = humemai.write_long_term_vertex(
        participant_label, "episodic", time_vertex
    )
    humemai.write_edge(participant_vertex, "has_cp", cp_vertex)

    situation = [bar for foo in data_point["situation"] for bar in foo]
    
    if situation:        
        situation_label = "situation_" + str(data_point["cp_num"])
        situation_vertex = humemai.write_long_term_vertex(
            situation_label, "episodic", time_vertex, {foo["type"]: foo["content"] for foo in situation}
        )
        humemai.write_edge(cp_vertex, "has_situation", situation_vertex)

        human_action = [bar for foo in data_point["HumanAction"] for bar in foo]
        robot_action = [bar for foo in data_point["RobotAction"] for bar in foo]

        # for idx, action in enumerate(data_point["HumanAction"]):
        #     action_label = "action_" + str(idx)
        #     action_vertex = humemai.write_long_term_vertex(
        #         action_label, "episodic", time_vertex, action
        #     )
        #     humemai.write_edge(participant_vertex, "has_action", action_vertex)

# Merge the duplicate nodes
humemai.merge_vertices_with_same_label("robot")

vertices = humemai.get_vertices_by_partial_label("participant")
for vertex in vertices:
    humemai.merge_vertices_with_same_label(vertex.label)

  0%|          | 1/211 [00:00<00:33,  6.23it/s]


AttributeError: 'list' object has no attribute 'items'

In [5]:
action

[{'type': 'action', 'content': 'Stand still in <location>'},
 {'type': 'location', 'content': '<Right> side of field'}]

In [ ]:
{foo["type"]: foo["content"] for foo in situation}

{'location': 'Top of rock pile', 'object': 'Brown rock'}

In [ ]:
situation

[{'type': 'location', 'content': 'Top of rock pile'},
 {'type': 'object', 'content': 'Brown rock'}]

In [ ]:
for action in data_point["HumanAction"]:
    print(action)

[{'type': 'action', 'content': 'Stand still in <location>'}, {'type': 'location', 'content': 'On top of <Object>'}, {'type': 'object', 'content': 'Large rock'}]
[]
[]


In [ ]:
from collections import Counter

# Get all vertex labels
labels = humemai.g.V().label().toList()

# Count the occurrences of each label
label_counts = Counter(labels)

# Filter labels that occur more than once
labels_more_than_once = {label: count for label, count in label_counts.items() if count > 1}

print(labels_more_than_once)


{'2024-08-29T15:36:08': 3, '2024-08-29T15:44:12': 2, '2024-08-29T15:48:34': 2, '2024-08-29T15:54:42': 3, '2024-08-29T16:10:21': 3, '2024-06-03T11:41:45': 2, '2024-06-03T12:05:43': 3, '2024-07-10T15:49:53': 2, '2024-07-23T12:17:49': 2, '2024-07-18T10:04:59': 3, '2024-07-18T10:26:55': 2, '2024-07-18T10:37:21': 3, '2024-07-22T09:47:40': 5, '2024-07-30T16:03:38': 3, '2024-08-07T10:41:02': 2, '2024-08-29T16:03:13': 3, '2024-08-23T11:11:30': 2, '2024-08-23T11:29:22': 3, '2024-05-07T11:26:11': 2, '2024-05-15T11:23:23': 2, '2024-05-15T11:34:16': 2, '2024-05-15T11:51:45': 3, '2024-05-31T10:37:11': 2, '2024-07-02T15:32:48': 2, '2024-07-02T15:52:00': 2, '2024-07-02T16:10:18': 2, '2024-07-03T10:51:18': 2, '2024-07-10T15:35:20': 3, '2024-07-10T15:53:45': 3, '2024-07-12T14:36:58': 2, '2024-07-22T09:53:10': 3, '2024-07-30T15:21:13': 2, '2024-07-30T15:30:18': 2, '2024-07-30T15:50:47': 3, '2024-07-30T16:00:03': 3, '2024-07-30T16:09:11': 3, '2024-08-07T10:23:02': 2, '2024-08-07T10:32:50': 3, '2024-08-29

In [ ]:
for foo in labels_more_than_once:
    print(foo)

2024-08-29T15:36:08
2024-08-29T15:44:12
2024-08-29T15:48:34
2024-08-29T15:54:42
2024-08-29T16:10:21
2024-06-03T11:41:45
2024-06-03T12:05:43
2024-07-10T15:49:53
2024-07-23T12:17:49
2024-07-18T10:04:59
2024-07-18T10:26:55
2024-07-18T10:37:21
2024-07-22T09:47:40
2024-07-30T16:03:38
2024-08-07T10:41:02
2024-08-29T16:03:13
2024-08-23T11:11:30
2024-08-23T11:29:22
2024-05-07T11:26:11
2024-05-15T11:23:23
2024-05-15T11:34:16
2024-05-15T11:51:45
2024-05-31T10:37:11
2024-07-02T15:32:48
2024-07-02T15:52:00
2024-07-02T16:10:18
2024-07-03T10:51:18
2024-07-10T15:35:20
2024-07-10T15:53:45
2024-07-12T14:36:58
2024-07-22T09:53:10
2024-07-30T15:21:13
2024-07-30T15:30:18
2024-07-30T15:50:47
2024-07-30T16:00:03
2024-07-30T16:09:11
2024-08-07T10:23:02
2024-08-07T10:32:50
2024-08-29T15:30:51
2024-05-15T11:13:40
2024-05-15T11:19:41
2024-06-03T11:54:49
2024-07-02T16:17:13
2024-07-10T15:28:26
2024-07-12T14:47:40
2024-08-07T10:08:27
2024-08-07T10:44:15
2024-05-15T11:45:01
2024-07-10T15:58:05
2024-07-18T10:44:02


In [ ]:
len(humemai.g.V().label().toList()), len(set(humemai.g.V().label().toList()))

(651, 561)